Load dependencies

In [1]:
import torchio as tio
from mctnet.lightning_modules import DataModule
import yaml
from yaml.loader import SafeLoader
import multiprocessing
import napari

with open('config.yaml', 'r') as f:
    config = yaml.load(f, Loader=SafeLoader)


data = DataModule(
    batch_size=config['batch_size'],
    train_val_ratio=config['train_val_ratio'],
    train_images_dir=config['train_images_dir'],
    train_labels_dir=config['train_labels_dir'],
    test_images_dir=config['test_images_dir'],
    test_labels_dir=config['test_labels_dir'],
    patch_size=config['patch_size'],
    samples_per_volume=config['samples_per_volume'],
    max_length=config['max_length'],
    num_workers=multiprocessing.cpu_count(),
    balanced_sampler=config['balanced_sampler'],
    label_suffix='corneas',
    sigma=config['starting_sigma'],
    learn_sigma=config['learn_sigma'],
    heatmap_max_length=25,
    balanced_sampler_length=9,
    histogram_landmarks_path=config['histogram_landmarks_path'],
)

Lazy load the images and setup everything else

In [10]:
data.setup()
images = data.subjects

/home/jake/projects/mctnet/venv/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(


Select an image and plot it

In [26]:
example_img = images[1]
viewr = napari.view_image(example_img.image.numpy(), name='original image')

/home/jake/projects/mctnet/venv/lib/python3.8/site-packages/napari/_vispy/vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


Now transform the image and plot it again

In [27]:
random_affine = tio.Compose([
    tio.RandomAffine(scales=0.3, degrees=360, p=1),
])
elastic_deformation = tio.Compose([
    tio.RandomElasticDeformation(p=1),
])
random_affined_img = random_affine(example_img)
elastic_deformed_img = elastic_deformation(example_img)

In [28]:
viewr.add_image(random_affined_img.image.numpy(), name='random affined')
viewr.add_image(elastic_deformed_img.image.numpy(), name='elastic deformed')

<Image layer 'elastic deformed' at 0x7f1a2111b280>